In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import census_key
import json

In [2]:
# year_list = ['2009', '2010','2011', '2012', '2013', '2014', '2015', '2016']

year_list = ['2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
fail_master = []

fuck_you_census = {
    '2009': '',
    '2010': 'acs/',
    '2011': 'acs/',
    '2012': 'acs/',
    '2013': 'acs/',
    '2014': 'acs/',
    '2015': 'acs/',
    '2016': 'acs/'
}


variable_conversion = {
    '2009': ['B19013_001E', 'B25064_001E'],
    '2010': ['B19013_001E', 'B25064_001E'],
    '2011': ['B19013_001E', 'B25064_001E'],
    '2012': ['B19013_001E', 'B25064_001E'],
    '2013': ['B19013_001E', 'B25064_001E'],
    '2014': ['B19013_001E', 'B25064_001E'],
    '2015': ['B19013_001E', 'B25031_001E'],
    '2016': ['B19013_001E', 'B25031_001E']
}

In [3]:
for year in year_list:
    try:
        census_test = requests.get(f'https://api.census.gov/data/{year}/acs/flows?get=FULL1_NAME,FULL2_NAME,MOVEDIN,MOVEDOUT,MOVEDNET' +
                                       f'&for=county:*&in=state:06&key=' + census_key).json()
        census_df = pd.DataFrame(census_test)
        census_df.columns = census_df.iloc[0]
        census_df = census_df[1:]
        census_df = census_df.rename(columns = {'FULL1_NAME': 'County',
                                                'FULL2_NAME': 'Origin',
                                                'MOVEDIN': f'{year}IN',
                                                'MOVEDOUT': f'{year}OUT',
                                                'MOVEDNET': f'{year}NET'
                                                })
        census_df = census_df[['County', 'Origin', f'{year}IN', f'{year}OUT', f'{year}NET']]
        
        economic_test = requests.get(f'https://api.census.gov/data/{year}/{fuck_you_census[year]}acs5?get=NAME,' + 
                                     f'{variable_conversion[year][0]},' +
                                     f'{variable_conversion[year][1]}' +
                                        '&for=county:*&in=state:06&key=' +
                                        census_key).json()
        economic_df = pd.DataFrame(economic_test)
        economic_df.columns = economic_df.iloc[0]
        economic_df = economic_df[1:]
        economic_df = economic_df.rename(columns = {f'{variable_conversion[year][0]}': f'{year}MED_INC',
                                                f'{variable_conversion[year][1]}': f'{year}RENT'
                                                })
        economic_df = economic_df[['NAME', f'{year}MED_INC', f'{year}RENT']]
        
        if year == '2016':
            census_master = census_df
            economic_master = economic_df
            
        else:
            census_master = census_master.merge(census_df, on = ['County', 'Origin'], how = 'left')
            economic_master = economic_master.merge(economic_df, on = 'NAME')
    
    except:
        fail_master.append(year)
        print(year+" FAILED")

In [4]:
census_master.to_csv('census_master.csv')

In [5]:
census_merged = pd.read_csv('census_master.csv')

In [6]:
census_merged.fillna(0)
census_merged = census_merged.drop(["Unnamed: 0", 'Origin'],axis=1)

In [7]:
census_group = census_merged.groupby('County').sum()
census_group = census_group.reset_index()

In [20]:
economic_master

,NAME,2016MED_INC,2016RENT,2015MED_INC,2015RENT,2014MED_INC,2014RENT,2013MED_INC,2013RENT,2012MED_INC,2012RENT,2011MED_INC,2011RENT,2010MED_INC,2010RENT,2009MED_INC,2009RENT
0,"Alameda County, California",79831,1432,75619,1367,73775,1325,72112,1289,71516,1265,70821,1228,69384,1188,68863,1166
1,"Alpine County, California",62375,1109,52917,None,61343,775,58636,1018,59931,755,59018,796,63478,913,54605,820
2,"Amador County, California",57032,1061,54171,1047,52964,1029,53684,1006,53462,1039,56180,1011,54758,1059,57347,1075
3,"Butte County, California",44366,936,43444,905,43165,899,43752,890,43339,885,42971,878,43170,861,42002,843
4,"Calaveras County, California",53502,1120,53233,1038,54936,1072,55295,1061,54686,1043,55256,1031,54971,948,57676,920
5,"Colusa County, California",54946,896,52168,869,50503,864,52158,923,52165,904,49558,885,48016,830,47150,762
6,"Contra Costa County, California",82881,1506,80185,1426,79799,1395,78756,1365,78187,1340,79135,1309,78385,1270,77838,1239
7,"Del Norte County, California",42363,818,40847,None,39302,843,37909,914,39626,865,37588,869,36118,845,38408,770
8,"El Dorado County, California",72586,1105,69584,1054,68507,1073,69297,1090,70117,1109,68815,1098,70000,1074,70765,1062
9,"Fresno County, California",45963,901,45233,886,45201,884,45563,875,45741,864,46903,850,46430,819,46230,801


In [8]:
census_final = census_group.merge(economic_master, left_on = 'County', right_on = 'NAME')

In [ ]:
census_final

In [9]:
census_final.to_csv('census_final.csv')